In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_copy
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_baseline
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_combined
/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_combined
/kaggle/input/Entertainment_Music/model_outputs/informal.rule_based
/kaggle/input/Entertainment_Music/model_outputs/informal.pbmt
/kaggle/input/Entertainment_Music/model_outputs/formal.nmt_copy
/kaggle/input/Entertainment_Music/model_outputs/formal.pbmt
/kaggle/input/Entertainment_Music/model_outputs/informal.nmt_baseline
/kaggle/input/Entertainment_Music/model_outputs/formal.rule_based
/kaggle/input/Entertainment_Music/tune/formal
/kaggle/input/Entertainment_Music/tune/informal.ref2
/kaggle/input/Entertainment_Music/tune/formal.ref2
/kaggle/input/Entertainment_Music/tune/formal.ref3
/kaggle/input/Entertainment_Music/tune/formal.ref1
/kaggle/input/Entertainment_Music/tune/informal
/kaggle/input/Entertainment_Music/tune

In [2]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.3 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [63]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [64]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-informal",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
     fp16=True,
    push_to_hub=True,
)

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

# Initialize Snowball stemmer
stemmer = SnowballStemmer("english")

# Initialize stopwords
stop_words = set(stopwords.words('english'))

# Function to apply stemming to a sentence using Snowball stemmer
def apply_stemming(sentence):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Function to replace terms with POS tags based on weights from the model
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)

def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [apply_stemming(sentence.strip()) for sentence in sentences]

# Paths to data files
train_formal_path_1 = "/kaggle/input/Entertainment_Music/train/formal"
train_informal_path_1 = "/kaggle/input/Entertainment_Music/train/informal"
test_formal_path_1 = "/kaggle/input/Entertainment_Music/test/formal"
test_informal_path_1 = "/kaggle/input/Entertainment_Music/test/informal"

train_formal_path_2 = "/kaggle/input/Family_Relationships/train/formal"
train_informal_path_2 = "/kaggle/input/Family_Relationships/train/informal"
test_formal_path_2 = "/kaggle/input/Family_Relationships/test/formal"
test_informal_path_2 = "/kaggle/input/Family_Relationships/test/informal"

X_train = (read_data(train_formal_path_1) + read_data(train_formal_path_2) +
           read_data(train_informal_path_1) + read_data(train_informal_path_2))
Y_train = [0] * (len(read_data(train_formal_path_1)) + len(read_data(train_formal_path_2))) + \
          [1] * (len(read_data(train_informal_path_1)) + len(read_data(train_informal_path_2)))  # Labels for training data

X_test = read_data(test_formal_path_1) + read_data(test_formal_path_2) + \
         read_data(test_informal_path_1) + read_data(test_informal_path_2)
Y_test = [0] * (len(read_data(test_formal_path_1)) + len(read_data(test_formal_path_2))) + \
         [1] * (len(read_data(test_informal_path_1)) + len(read_data(test_informal_path_2)))  # Labels for test data


# Initialize TfidfVectorizer with n-grams
vectorizer = TfidfVectorizer(max_features=200000, ngram_range=(1, 3), min_df=1, max_df=0.85)


# Initialize StandardScaler for scaling the data
scaler = StandardScaler(with_mean=False)

# Create a Pipeline for preprocessing and model training
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', LogisticRegression( max_iter=5000))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Extract feature names from the vectorizer
# feature_names = vectorizer.get_feature_names()
feature_names = vectorizer.get_feature_names_out()


# Extract coefficients (weights) from the logistic regression model
weights = dict(zip(feature_names, pipeline.named_steps['classifier'].coef_[0]))

# Set threshold for replacing terms with POS tags
# threshold = 0.2

# Replace terms with POS tags in the training data itself
X_train_transformed = [replace_with_pos_tags(X_train[i], weights, not(Y_train[i])) for i  in range(len(X_train))]

# Print out the original and transformed sentences
# for original_sentence, transformed_sentence in zip(X_train, X_train_transformed):
#     print("Original Sentence:", original_sentence)
#     print("Transformed Sentence:", transformed_sentence)
#     print()  # Add a newline for clarity

# Make predictions on the test data
predicted_probabilities = pipeline.predict_proba(X_test)

# Define a new threshold
new_threshold = 0.42  # Example threshold, you can adjust it according to your needs

# Modify predictions based on the new threshold
predicted_labels = (predicted_probabilities[:, 1] >= new_threshold).astype(int)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)
print("Accuracy:", accuracy)

# Compute confusion matrix
cm = confusion_matrix(Y_test, predicted_labels)
print("Confusion Matrix:")
print(cm)


Accuracy: 0.8354299855640338
Confusion Matrix:
[[1665  436]
 [ 362 2386]]


In [7]:
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)


In [8]:
def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [sentence.strip() for sentence in sentences]

In [9]:
X_train = (read_data(train_formal_path_1) + read_data(train_formal_path_2) +
           read_data(train_informal_path_1) + read_data(train_informal_path_2))
Y_train = [0] * (len(read_data(train_formal_path_1)) + len(read_data(train_formal_path_2))) + \
          [1] * (len(read_data(train_informal_path_1)) + len(read_data(train_informal_path_2)))  # Labels for training data

X_test = read_data(test_formal_path_1) + read_data(test_formal_path_2) + \
         read_data(test_informal_path_1) + read_data(test_informal_path_2)
Y_test = [0] * (len(read_data(test_formal_path_1)) + len(read_data(test_formal_path_2))) + \
         [1] * (len(read_data(test_informal_path_1)) + len(read_data(test_informal_path_2)))

In [11]:
X_train_transformed = [replace_with_pos_tags(X_train[i], weights, not(Y_train[i])) for i  in range(len(X_train))]


In [ ]:
new=replace_with_pos_tags("There's nothing he needs to change.", weights, True)

new

In [ ]:
X_test=[apply_stemming(sentence) for sentence in X_test]

In [13]:
# X_test

In [ ]:
predicted_probabilities = pipeline.predict_proba(X_test)

# Define a new threshold
new_threshold = 0.45  # Example threshold, you can adjust it according to your needs

# Modify predictions based on the new threshold
predicted_labels = (predicted_probabilities[:, 1] >= new_threshold).astype(int)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)
print("Accuracy:", accuracy)

# Compute confusion matrix
cm = confusion_matrix(Y_test, predicted_labels)
print("Confusion Matrix:")
print(cm)

In [14]:
pipeline


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.85, max_features=200000,
                                 ngram_range=(1, 3))),
                ('classifier', LogisticRegression(max_iter=5000))])

In [15]:
weights["crap"]

3.879674124709792

In [16]:
X_test_transformed = [replace_with_pos_tags(X_test[i], weights, not(Y_test[i])) for i  in range(len(X_test))]

In [ ]:
# X_test_transformed

In [17]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00a 0:00:01


In [18]:
from evaluate import load
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3e91e6b7d028f952323eac9b05f3264c8a5fcbc363f447a35274a94d0856a004
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [19]:
metric1 = load("rouge")

In [20]:
metric2 = load("bleu")

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [22]:
def read_data(file_path):
    with open(file_path, "r", encoding="latin-1") as file:
        sentences = file.readlines()
    return [sentence.strip() for sentence in sentences]


# Paths to data files
train_formal_path_1 = "/kaggle/input/Entertainment_Music/train/formal"
train_informal_path_1 = "/kaggle/input/Entertainment_Music/train/informal"
test_formal_path_1 = "/kaggle/input/Entertainment_Music/test/formal"
test_informal_path_1 = "/kaggle/input/Entertainment_Music/test/informal"

train_formal_path_2 = "/kaggle/input/Family_Relationships/train/formal"
train_informal_path_2 = "/kaggle/input/Family_Relationships/train/informal"
test_formal_path_2 = "/kaggle/input/Family_Relationships/test/formal"
test_informal_path_2 = "/kaggle/input/Family_Relationships/test/informal"

X_train_formal = (read_data(train_formal_path_1) + read_data(train_formal_path_2) )
X_train_informal = read_data(train_informal_path_1) + read_data(train_informal_path_2)


X_test_formal = read_data(test_formal_path_1) + read_data(test_formal_path_2)
X_test_informal = read_data(test_informal_path_1) + read_data(test_informal_path_2)


In [23]:
X_train_informal_transformed=X_train_transformed[len(X_train)//2:]

In [24]:
X_test_informal_transformed=X_test_transformed[len(X_test_formal):]

In [25]:
train_informal_df = pd.DataFrame({ 'POS': X_train_informal_transformed, 'informal': X_train_informal,})

In [27]:
train_informal_df

,POS,informal
0,the NN The In-Laws not exactly a holiday NN bu...,the movie The In-Laws not exactly a holiday mo...
1,that page did not VB me viroses ( i VBP ),that page did not give me viroses(i think)
2,"of NN i be wachin it evry NN , my JJ charachte...","of corse i be wachin it evry day, my fav chara..."
3,runescape.com ( my NNS VB it ) & funbrain.com ...,runescape.com (my kids love it) & funbrain.com...
4,Is he RB ? He was on Late Night with Conan O'B...,Is he gay?He was on Late Night with Conan O'Br...
...,...,...
104557,of NN it VBZ on what NN u r looking for .,of corse it depends on what relation u r looki...
104558,Wear a sign that says NNP !,Wear a sign that says Hi!
104559,"I do n't VB in VBG games , I VBP when NNS do t...","I don't believe in playing games, I hate when ..."
104560,( or w/e ) p.s NN how JJ r u ?,(or w/e) p.s gurl how old r u ?


In [34]:
test_informal_df = pd.DataFrame({ 'POS': X_test_informal_transformed, 'informal': X_test_informal,})

In [29]:
test_formal_df

,POS,informal
0,Is Any Baby Really A NNP .,Is Any Baby Really A Freak.
1,"aspen NN has he JJS NN NNS , you sit all over ...","aspen colorado has he best music festivals, yo..."
2,You can VB almost NN on ebay !,You can get almost anything on ebay!
3,everybody is Dying to VB in,everybody is Dying to get in
4,not idiots IN 50 cent and his JJ NNP unit.thos...,not idiots like 50 cent and his whole Gay unit...
...,...,...
2743,NNS have a problem with secrets .,guys have a problem with secrets.
2744,I had someone NN JJ with me because I was stil...,I had someone get nasty with me because I was ...
2745,So how do i just VB him,So how do i just kiss him
2746,I am the NN of honor for my only NN and I VBP ...,I am the maid of honor for my only sister and ...


In [32]:
train_informal_df

,POS,informal
0,the NN The In-Laws not exactly a holiday NN bu...,the movie The In-Laws not exactly a holiday mo...
1,that page did not VB me viroses ( i VBP ),that page did not give me viroses(i think)
2,"of NN i be wachin it evry NN , my JJ charachte...","of corse i be wachin it evry day, my fav chara..."
3,runescape.com ( my NNS VB it ) & funbrain.com ...,runescape.com (my kids love it) & funbrain.com...
4,Is he RB ? He was on Late Night with Conan O'B...,Is he gay?He was on Late Night with Conan O'Br...
...,...,...
104557,of NN it VBZ on what NN u r looking for .,of corse it depends on what relation u r looki...
104558,Wear a sign that says NNP !,Wear a sign that says Hi!
104559,"I do n't VB in VBG games , I VBP when NNS do t...","I don't believe in playing games, I hate when ..."
104560,( or w/e ) p.s NN how JJ r u ?,(or w/e) p.s gurl how old r u ?


In [35]:
test_informal_df

,POS,informal
0,Is Any Baby Really A NNP .,Is Any Baby Really A Freak.
1,"aspen NN has he JJS NN NNS , you sit all over ...","aspen colorado has he best music festivals, yo..."
2,You can VB almost NN on ebay !,You can get almost anything on ebay!
3,everybody is Dying to VB in,everybody is Dying to get in
4,not idiots IN 50 cent and his JJ NNP unit.thos...,not idiots like 50 cent and his whole Gay unit...
...,...,...
2743,NNS have a problem with secrets .,guys have a problem with secrets.
2744,I had someone NN JJ with me because I was stil...,I had someone get nasty with me because I was ...
2745,So how do i just VB him,So how do i just kiss him
2746,I am the NN of honor for my only NN and I VBP ...,I am the maid of honor for my only sister and ...


In [67]:
import pandas as pd

# Assuming you already have X_train_formal_transformed and X_train_formal defined
train_informal_df = pd.DataFrame({'POS': X_train_informal_transformed, 'informal': X_train_informal})
# train_informal_df = pd.DataFrame({ 'POS': X_train_informal_transformed, 'informal': X_train_informal,})

def preprocess_function(row):
    inputs = row["POS"] + "</s>"
    model_inputs = tokenizer(inputs, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=row["informal"], truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset_informal_train = train_informal_df.apply(preprocess_function, axis=1)
tokenized_dataset_informal_test = test_informal_df.apply(preprocess_function, axis=1)


In [68]:
from transformers import AutoModelForSeq2SeqLM


In [69]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(tokenized_dataset_informal_train, test_size=0.2, random_state=42)


In [70]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [72]:
import nltk
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    preds=[apply_stemming(sentence) for sentence in  decoded_preds]

    # Use logistic regression model to predict formality of decoded sentences
    predicted_formality = pipeline.predict(preds)

    # Calculate accuracy based on logistic regression model's predictions
    total_samples = len(decoded_preds)
    correct_predictions = sum(predicted_formality == 1)  # Count how many were predicted as formal (0)
    accuracy_log_reg = correct_predictions / total_samples

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Add accuracy based on logistic regression model's predictions
    result["accuracy_log_reg"] = accuracy_log_reg
    
    print(result)

    return {k: round(v, 4) for k, v in result.items()}


In [73]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')

In [74]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [48]:
print(train_dataset1[:10])

[{'input_ids': [852, 3, 88, 19, 584, 19179, 91, 28, 3, 9, 9269, 3, 17235, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [852, 3, 88, 19, 352, 91, 28, 3, 9, 9269, 4024, 5, 1]}, {'input_ids': [3, 2, 834, 2, 27, 584, 11165, 12, 3, 22086, 14883, 11113, 8, 445, 9082, 28, 82, 446, 683, 19665, 230, 11, 258, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3, 2, 834, 2, 27, 333, 12, 1605, 14883, 11113, 8, 1908, 32, 107, 28, 82, 385, 19665, 230, 11, 258, 5, 1]}, {'input_ids': [11, 24, 3, 31, 7, 59, 3, 9, 446, 683, 3, 17235, 3, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [11, 24, 31, 7, 59, 3, 9, 1282, 589, 5, 1]}, {'input_ids': [27, 584, 14594, 11790, 15, 63, 3, 88, 3, 31, 7, 78, 5295, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 6528, 11790, 15, 63, 3, 88, 31, 7, 78, 5295, 1]}, {'input_i

In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [75]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Accuracy Log Reg
1,0.951300,0.817946,79.305300,64.409000,79.184800,79.179700,14.975000,0.854500
2,0.864900,0.757735,80.204700,66.004000,80.101400,80.095800,14.971200,0.866500


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 79.30532436342062, 'rouge2': 64.4089665138969, 'rougeL': 79.18476650749918, 'rougeLsum': 79.1797038745427, 'gen_len': 14.975039449146465, 'accuracy_log_reg': 0.8545402381293932}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'rouge1': 80.20468205422158, 'rouge2': 66.00398643469936, 'rougeL': 80.10139414255583, 'rougeLsum': 80.09578572125537, 'gen_len': 14.971214077368144, 'accuracy_log_reg': 0.8664945249366423}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0,

In [86]:
trainer.push_to_hub("Pushparaj20/informal_transfer")

events.out.tfevents.1713877346.cbf3c2509588.34.3:   0%|          | 0.00/20.0k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Pushparaj20/t5-small-informal/commit/27c96c85a91cb399cfc284213d75fa6b7175dff3', commit_message='Pushparaj20/informal_transfer', commit_description='', oid='27c96c85a91cb399cfc284213d75fa6b7175dff3', pr_url=None, pr_revision=None, pr_num=None)

In [88]:
import joblib
weights_file = "weight.pkl"
joblib.dump(weights, weights_file)

['weight.pkl']

In [ ]:
import joblib

# Save the entire pipeline (including vectorizer and classifier)
# joblib.dump(pipeline, 'formality_model.pkl')

# Save the logistic regression model weights
joblib.dump(pipeline.named_steps['classifier'].coef_, 'weights.pkl')


In [137]:
import os
import requests

# Define the repository and file names
repo_name = "Pushparaj20/t5-base-finetuned"  # Replace with your actual username and repository name
file_name = "weight.pkl"

# Construct the URL to fetch the file from the repository
file_url = f"https://huggingface.co/{repo_name}/resolve/main/{file_name}"

# Define the directory where you want to save the file locally
output_dir = "logreg"  # Choose a directory name

# Make a GET request to download the file
response = requests.get(file_url)

# Check if the request was successful
if response.status_code == 200:
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Write the file content to disk
    with open(os.path.join(output_dir, file_name), "wb") as file:
        file.write(response.content)
    print(f"File '{file_name}' downloaded successfully to '{output_dir}'.")
else:
    print(f"Failed to download '{file_name}' from '{repo_name}'. Status code: {response.status_code}")


File 'weight.pkl' downloaded successfully to 'logreg'.


In [ ]:
def replace_with_pos_tags(sentence, weights, feature_names, is_formal):
    stemmer = PorterStemmer()
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 5

    # Sort stemmed tokens by weight
    sorted_stemmed_tokens = sorted(stemmed_tokens, key=lambda x: abs(weights[0, feature_names.index(x)]), reverse=True)

    # Extract N stemmed terms with highest absolute weights
    top_N_stemmed_tokens = sorted_stemmed_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_stemmed_tokens:
            weight = abs(weights[0, feature_names.index(stemmed_token)])
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)



sentence = "I don't think that page gave me viruses."
weights_file = "logregweights.pkl"  # Path to the weights file within the logreg directory
is_formal = True  # Example value

# Load the weights from the file
weights = joblib.load(weights_file)

# Ensure that you also pass the feature_names argument
transformed_sentence = replace_with_pos_tags(sentence, weights, feature_names, is_formal)
print("Transformed Sentence:", transformed_sentence)

In [ ]:
weights_file


In [ ]:
import joblib

# Load the weights from the file
loaded_weights = joblib.load('new.pkl')

# Print the type and shape of the loaded weights
print("Type of loaded weights:", type(loaded_weights))
print("Shape of loaded weights:", loaded_weights.shape)

# Print the loaded weights
print("Loaded weights:", loaded_weights)

In [ ]:
pipeline

In [ ]:
import joblib

# Load the weights from the file
loaded_weights = joblib.load('logreg_weights.pkl')

# Print the type of loaded weights and the first few entries
print(type(loaded_weights))
print(list(loaded_weights.items())[:10])  # Print the first 10 items


# Check if the structure is similar to the weights dictionary


In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer

# Load the weights from the file
weights_file = 'logreg/weight.pkl'
loaded_weights = joblib.load(weights_file)

# Define a sample sentence
sentence = "s."

# Specify the value of is_formal
is_formal = True  # Example value
stemmer = SnowballStemmer("english")

# Define the function to replace words with POS tags
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)

# Call the function with loaded weights, sample sentence, and is_formal value
transformed_sentence = replace_with_pos_tags(sentence, loaded_weights, is_formal)

# Print the transformed sentence
print("Transformed Sentence:", transformed_sentence)


In [ ]:
weights_file = "weight.pkl"
joblib.dump(weights, weights_file)

In [ ]:
import os
import requests

# Define the repository and file names
repo_name = "Pushparaj20/t5-base-finetuned"  # Replace with your actual username and repository name
file_name = "weight.pkl"

# Construct the URL to fetch the file from the repository
file_url = f"https://huggingface.co/{repo_name}/resolve/main/{file_name}"

# Define the directory where you want to save the file locally
output_dir = "logreg"  # Choose a directory name

# Make a GET request to download the file
response = requests.get(file_url)

# Check if the request was successful
if response.status_code == 200:
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Write the file content to disk
    with open(os.path.join(output_dir, file_name), "wb") as file:
        file.write(response.content)
    print(f"File '{file_name}' downloaded successfully to '{output_dir}'.")
else:
    print(f"Failed to download '{file_name}' from '{repo_name}'. Status code: {response.status_code}")


In [176]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-base-finetuned")
from transformers import AutoTokenizer

# Load the tokenizer from the model hub
tokenizer = AutoTokenizer.from_pretrained("Pushparaj20/t5-base-finetuned")


In [170]:
import joblib
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer

# Load the weights from the file
weights_file = 'logreg/weight.pkl'
loaded_weights = joblib.load(weights_file)

# Define a sample sentence
sentence =  "OMG! It's finger-lickin' good."

# Specify the value of is_formal
is_formal = True  # Example value
stemmer = SnowballStemmer("english")

# Define the function to replace words with POS tags
def replace_with_pos_tags(sentence, weights, is_formal):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
    pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
    replaced_sentence = []

    # Calculate N as described
    N = len(stemmed_tokens) // 4

    # Filter out stop words and stem them
    tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

    # Sort stemmed tokens by weight
    sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

    # Extract N tokens with highest absolute weights
    top_N_tokens = sorted_tokens[:N]

    for token, tag in pos_tags:
        stemmed_token = stemmer.stem(token)
        if stemmed_token in top_N_tokens:
            weight = abs(weights.get(stemmed_token, 0))
            if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                replaced_sentence.append(tag)
            else:
                replaced_sentence.append(token)
        else:
            replaced_sentence.append(token)

    return ' '.join(replaced_sentence)

# Call the function with loaded weights, sample sentence, and is_formal value
transformed_sentence = replace_with_pos_tags(sentence, loaded_weights, is_formal)

# Print the transformed sentence
print("Transformed Sentence:", transformed_sentence)


Transformed Sentence: NN ! It 's finger-lickin ' JJ .


In [177]:

input_ids = tokenizer(transformed_sentence, return_tensors="pt").input_ids
print(input_ids)

# Generate predictions
output = model.generate(input_ids)
b=output.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

tensor([[    3, 17235,     3,    55,    94,     3,    31,     7,  8425,    18,
         10095,    77,     3,    31,   446,   683,     3,     5,     1]])


"Hey! It's finger-lickin' hot."